In [88]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import nltk

In [89]:
df_tweets = pd.read_csv('umk_tweets_202112_clean.csv')
df_tweets

,tweet,userid
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658
2,umk yo regone mundak maneh bos,1572322873
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136
4,sumpah umk malang segini iya mending nyari ker...,181023564
...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592
1599,umk solo rp ribu dinilai layak,2392460208
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576


In [90]:
df_tweets_model = pd.read_csv('dataset_komentar_instagram_cyberbullying.csv')
df_tweets_model

,Id,Sentiment,Instagram Comment Text
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...,...
395,396,positive,Bangga sama suami yg selalu ingat istri disela...
396,397,positive,Apaoun pekerjaannya yg penting halal u tuk men...
397,398,positive,Gojek itu mayoritas pegangguran yang lama gak ...
398,399,positive,<USERNAME> aslinya cantik dan ayu loh mbak kr...


In [91]:
df_tweets_model.rename(columns={"Instagram Comment Text":"Text"}, inplace=True)
df_tweets_model.drop(columns=["Id"],inplace=True,)
df_tweets_model.replace({"Sentiment":{"negative":0, "positive":1}}, inplace=True)

In [92]:
df_tweets_model

,Sentiment,Text
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."
...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...


In [93]:
df_tweets['clean_tweet'] = df_tweets['tweet'].str.lower()

df_tweets['clean_tweet'] = df_tweets['clean_tweet'].str.replace(r'\d+', '', regex=True)

df_tweets['clean_tweet'] = df_tweets['clean_tweet'].str.translate(str.maketrans("","",string.punctuation))
df_tweets['clean_tweet'] = df_tweets['clean_tweet'].str.strip()

df_tweets['clean_tweet'] = df_tweets['clean_tweet'].str.split()

df_tweets

,tweet,userid,clean_tweet
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,"[umk, sragen, ditetapkan, rp, rp, sayang, tere..."
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,"[umk, kota, bekasi, ump, dki, jakarta, yak, te..."
2,umk yo regone mundak maneh bos,1572322873,"[umk, yo, regone, mundak, maneh, bos]"
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,"[umk, kabupatenkota, jawa, resmi, diumumkan, k..."
4,sumpah umk malang segini iya mending nyari ker...,181023564,"[sumpah, umk, malang, segini, iya, mending, ny..."
...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,"[ngelihat, gaji, dihitung, make, idr, susah, n..."
1599,umk solo rp ribu dinilai layak,2392460208,"[umk, solo, rp, ribu, dinilai, layak]"
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,"[bicaranya, tidak, sesuai, kenyataan, jakarta,..."
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,"[dikabarkan, persen, daftar, umk, provinsi, ba..."


In [94]:
def emoji_cleaner(data):
    word_emoji = data
    word_list = []
    
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                    "]+", flags=re.UNICODE)
    
    for word in word_emoji:
        emoji_pattern.sub(r'', word) 
        word_list.append(word)
    data = ' '.join(word_list)
    return data

df_tweets['clean_tweet'] = df_tweets['clean_tweet'].apply(lambda x: emoji_cleaner(x))
df_tweets

,tweet,userid,clean_tweet
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss😭
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...
...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya tidak sesuai kenyataan jakarta warga...
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...


In [102]:
with open('formalizationDict.txt') as f:
    df_slang = list(f)

slang_list = df_slang
def tokenizing(data):
    data = word_tokenize(data)
    return data

def slang_cleaner(data, slang_list:list=[]):
    word_token = tokenizing(data)
    word_list = []
    
    for word in word_token:
        for slang in slang_list:
            if word.lower() in slang[0].lower():
                word = slang[1]
                break
        word_list.append(word)
        
    data = ' '.join(word_list)
    return data

df_tweets['clean_tweet'] = df_tweets['clean_tweet'].apply(lambda x: slang_cleaner(x, slang_list))

In [103]:
df_tweets

,tweet,userid,clean_tweet
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss😭
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...
...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya tidak sesuai kenyataan jakarta warga...
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...


In [106]:
df_tweets_model['clean_tweet'] = df_tweets_model['Text'].str.lower()

df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].str.replace(r'\d+', '', regex=True)

df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].str.translate(str.maketrans("","",string.punctuation))
df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].str.strip()

df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].str.split()

df_tweets_model

,Sentiment,Text,clean_tweet
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,"[username, tolol, gak, ada, hubungan, nya, keg..."
1,0,Geblek lo tata...cowo bgt dibela2in balikan......,"[geblek, lo, tatacowo, bgt, dibelain, balikanh..."
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...,"[kmrn, termewek, skr, lengket, lg, duhhh, kok,..."
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ...","[intinya, kalau, kesel, dengan, att, nya, gaus..."
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...","[hadewwwww, permpuan, itu, lgsakit, jiwaknp, h..."
...,...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...,"[bangga, sama, suami, yg, selalu, ingat, istri..."
396,1,Apaoun pekerjaannya yg penting halal u tuk men...,"[apaoun, pekerjaannya, yg, penting, halal, u, ..."
397,1,Gojek itu mayoritas pegangguran yang lama gak ...,"[gojek, itu, mayoritas, pegangguran, yang, lam..."
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...,"[username, aslinya, cantik, dan, ayu, loh, mba..."


In [107]:
def emoji_cleaner(data):
    word_emoji = data
    word_list = []
    
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                    "]+", flags=re.UNICODE)
    
    for word in word_emoji:
        emoji_pattern.sub(r'', word) 
        word_list.append(word)
    data = ' '.join(word_list)
    return data

df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].apply(lambda x: emoji_cleaner(x))
df_tweets_model

,Sentiment,Text,clean_tweet
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,username tolol gak ada hubungan nya keguguran ...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......,geblek lo tatacowo bgt dibelain balikanhadewwn...
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek skr lengket lg duhhh kok labil b...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ...",intinya kalau kesel dengan att nya gausah ke a...
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",hadewwwww permpuan itu lgsakit jiwaknp harus d...
...,...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...,bangga sama suami yg selalu ingat istri disela...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...,apaoun pekerjaannya yg penting halal u tuk men...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...,gojek itu mayoritas pegangguran yang lama gak ...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...,username aslinya cantik dan ayu loh mbak krn a...


In [108]:
with open('formalizationDict.txt') as f:
    df_slang = list(f)

slang_list = df_slang
def tokenizing(data):
    data = word_tokenize(data)
    return data

def slang_cleaner(data, slang_list:list=[]):
    word_token = tokenizing(data)
    word_list = []
    
    for word in word_token:
        for slang in slang_list:
            if word.lower() in slang[0].lower():
                word = slang[1]
                break
        word_list.append(word)
        
    data = ' '.join(word_list)
    return data

df_tweets_model['clean_tweet'] = df_tweets_model['clean_tweet'].apply(lambda x: slang_cleaner(x, slang_list))

In [109]:
df_tweets_model

,Sentiment,Text,clean_tweet
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...,username tolol gak ada hubungan nya keguguran ...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......,geblek lo tatacowo bgt dibelain balikanhadewwn...
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...,kmrn termewek skr lengket lg duhhh kok labil b...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ...",intinya kalau kesel dengan att nya gausah ke a...
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha...",hadewwwww permpuan itu lgsakit jiwaknp harus d...
...,...,...,...
395,1,Bangga sama suami yg selalu ingat istri disela...,bangga sama suami yg selalu ingat istri disela...
396,1,Apaoun pekerjaannya yg penting halal u tuk men...,apaoun pekerjaannya yg penting halal \t tuk me...
397,1,Gojek itu mayoritas pegangguran yang lama gak ...,gojek itu mayoritas pegangguran yang lama gak ...
398,1,<USERNAME> aslinya cantik dan ayu loh mbak kr...,username aslinya cantik dan ayu loh mbak krn a...


In [110]:
vectorizer = TfidfVectorizer (max_features=2500)
model_g = GaussianNB()

In [112]:
v_data = vectorizer.fit_transform(df_tweets_model['clean_tweet']).toarray()

In [115]:
print(vectorizer.fit_transform(df_tweets_model['clean_tweet']))

  (0, 1192)	0.1935793469803282
  (0, 1676)	0.1935793469803282
  (0, 129)	0.16542822958912562
  (0, 946)	0.1935793469803282
  (0, 2462)	0.08305944732631089
  (0, 2136)	0.11723578442864521
  (0, 2143)	0.1935793469803282
  (0, 977)	0.1935793469803282
  (0, 1998)	0.09348114133099904
  (0, 1727)	0.1316756634767781
  (0, 2461)	0.1935793469803282
  (0, 585)	0.1935793469803282
  (0, 1124)	0.1935793469803282
  (0, 244)	0.1935793469803282
  (0, 361)	0.12607421475563316
  (0, 1292)	0.2556605750823826
  (0, 2486)	0.06417053125208308
  (0, 2241)	0.1935793469803282
  (0, 869)	0.18112227908764844
  (0, 1739)	0.12441311240018046
  (0, 568)	0.1598267808679807
  (0, 1083)	0.1935793469803282
  (0, 1681)	0.26573051635982553
  (0, 877)	0.3622445581752969
  (0, 13)	0.19317421639621707
  :	:
  (398, 46)	0.14224896621729516
  (398, 1324)	0.163266139610513
  (398, 161)	0.1299547079111083
  (398, 1205)	0.2844979324345903
  (398, 1294)	0.15306383981415644
  (398, 523)	0.07919744603764162
  (398, 1374)	0.11699401

In [116]:
X_train, X_test, y_train, y_test = train_test_split(v_data, df_tweets_model['Sentiment'], test_size=0.2, random_state=18)

model_g.fit(X_train,y_train)

GaussianNB()

In [117]:
y_preds = model_g.predict(X_test)

 

print(confusion_matrix(y_test,y_preds))

print(classification_report(y_test,y_preds))

print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[36  6]
 [ 6 32]]
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        42
           1       0.84      0.84      0.84        38

    accuracy                           0.85        80
   macro avg       0.85      0.85      0.85        80
weighted avg       0.85      0.85      0.85        80

nilai akurasinya adalah  0.85


In [118]:
v_data = vectorizer.transform(df_tweets["clean_tweet"]).toarray()

y_classification = model_g.predict(v_data)

 

df_tweets["sentiment"] = list(y_classification)

In [119]:
df_tweets

,tweet,userid,clean_tweet,sentiment
0,umk sragen ditetapkan rp rp sayang terendah so...,949224511508955136,umk sragen ditetapkan rp rp sayang terendah so...,0
1,umk kota bekasi ump dki jakarta yak terobosss😭,1259379786566086658,umk kota bekasi ump dki jakarta yak terobosss😭,1
2,umk yo regone mundak maneh bos,1572322873,umk yo regone mundak maneh bos,1
3,umk kabupatenkota jawa resmi diumumkan kota se...,949224511508955136,umk kabupatenkota jawa resmi diumumkan kota se...,0
4,sumpah umk malang segini iya mending nyari ker...,181023564,sumpah umk malang segini iya mending nyari ker...,0
...,...,...,...,...
1598,ngelihat gaji dihitung make idr susah nyentuh ...,1200035940372078592,ngelihat gaji dihitung make idr susah nyentuh ...,1
1599,umk solo rp ribu dinilai layak,2392460208,umk solo rp ribu dinilai layak,0
1600,bicaranya tidak sesuai kenyataan jakarta warga...,984001438408953857,bicaranya tidak sesuai kenyataan jakarta warga...,1
1601,dikabarkan persen daftar umk provinsi banten c...,1243113157066776576,dikabarkan persen daftar umk provinsi banten c...,0


In [124]:
tweets = [word for tweet in df_tweets['clean_tweet'] for word in tweet]

In [125]:
fqdist = FreqDist(tweets)

print(fqdist)

<FreqDist with 3572 samples and 16904 outcomes>


In [126]:
most_common_word = fqdist.most_common(20)

print(most_common_word)

[('umk', 1633), ('jawa', 589), ('tidak', 339), ('buruh', 320), ('kota', 253), ('gubernur', 239), ('timur', 237), ('barat', 236), ('rp', 197), ('upah', 173), ('kabupaten', 169), ('daftar', 156), ('kenaikan', 156), ('gaji', 146), ('besaran', 140), ('tertinggi', 124), ('daerah', 120), ('minimum', 117), ('kabupatenkota', 103), ('banten', 102)]


In [127]:
result = pd.Series(nltk.ngrams(tweets, 2)).value_counts()[:20]
print (result)

(jawa, timur)               232
(jawa, barat)               227
(umk, jawa)                 169
(upah, minimum)             116
(kenaikan, umk)             102
(besaran, umk)               93
(gubernur, jawa)             77
(daftar, umk)                73
(ridwan, kamil)              64
(umk, kabupaten)             64
(gaji, umk)                  63
(umk, kota)                  56
(rp, ribu)                   55
(penetapan, umk)             54
(daftar, lengkap)            48
(kota, bekasi)               47
(umk, kabupatenkota)         47
(tetapkan, umk)              45
(umk, tidak)                 41
(minimum, kabupatenkota)     41
dtype: int64
